In [26]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10)
import numpy as np
import uuid

In [2]:
def convert(s):
    if s is np.nan:
        return np.nan
    try:
        return float(s)
    except ValueError:
        num, denom = s.split('/')
        return float(num) / float(denom)
    
def convert_impact(s):
    if s is np.nan:
        return np.nan
    try:
        return float(s)
    except ValueError:
        return 0.


#ftr = [1, 1/60]
def convert_work_time(s):
    ftr = [1, 1/60]
    try:
        return sum([a*b for a,b in zip(ftr, map(int,s.split(':')))])
    except:# (ValueError):
        return np.nan


def eval_mean_sum(df):
    df = df.copy(deep=False)
    means = []
    sums = []
    for index, row in df.iterrows():
        means.append(0)
        sums.append(0)
        days_count = 0
        entries = np.array([])
        exits = np.array([])
        for _, val in row.iteritems():
            if val == '-' or len(val.split("\n")) != 4:
                continue
            days_count += 1
            entry = convert_work_time(val.split("\n")[0])
            exit = convert_work_time(val.split("\n")[1])
            entries = np.append(entries, entry)
            exits = np.append(exits, exit)

        if days_count == 0:
            means[index] = 0
            sums[index] = 0
            continue
        
        if np.isnan(np.nanmean(entries)):
            mean_entry = 8.5
        else:
            mean_entry = np.nanmean(entries)
        if np.isnan(np.nanmean(exits)):
            mean_exit = 17
        else:
            mean_exit = np.nanmean(exits)
        
        means[index] = abs(mean_exit - mean_entry)
        sums[index] = means[index] * days_count

    df['Ср продолжительность в УрГУПС'] = pd.Series(means)
    df['Общая продолжительность в УрГУПС'] = pd.Series(sums)
    df = df.drop(columns=df.columns[1:-2])
    return df


def convert_business_trip(s):
    if '(ин)' in s:
        return 'Иностр'
    if 'Москва' in s:
        return 'Москва'
    return 'Россия'


def convert_conferences(s):
    if 'ургупс' in s.lower() or 'уральский государственный университет путей сообщения' in s.lower():
        return 'в ургупс'
    return 'НЕ в ургупс'

<font size="20">Данные из отдела кадров</font>

In [3]:
HR = pd.read_excel('Работники241019 с внутренними.xlsx', sheet_name='Данные')
HR = HR.drop(columns=['Общий стаж лет', 'Педагогический стаж лет', 'Отрасль науки'])
HR = HR.rename(columns={'Сотрудник': 'ФИО', 
                        'Физическое лицо.Внешние руководители/специалисты стаж (Физические лица)': 'Дата приема внешнее место работы', 
                        'Физическое лицо.Внешние руководители/специалисты (Физические лица)': 'Внешнее совместительство'})
HR['ФИО'] = HR['ФИО'].str.title()
HR['ФИО'] = HR['ФИО'].replace('\s+', ' ', regex=True)
HR['ФИО'] = HR['ФИО'].replace('ё', 'е', regex=True)
HR['ФИО'] = HR['ФИО'].str.replace('Осн.', '').str.strip('()').str.strip()

external_work = HR[HR['Вид занятости'] == 'Внешнее совместительство'][['ФИО', 'Внешнее совместительство', 'Дата приема внешнее место работы']]
external_work['Внешнее совместительство'] = external_work['Внешнее совместительство'].str.lower()
searchfor = ['начальник', 'директор', 'руководител', 'главн']
external_work['Внешнее совместительство'] = external_work['Внешнее совместительство'].str.contains('|'.join(searchfor))
external_work['Внешнее совместительство'] = external_work['Внешнее совместительство'].map({np.nan: 'есть',
                                                                                           False: 'есть_специалист',
                                                                                           True: 'есть_начальник_директор_руководитель'})
HR = HR.drop(columns=['Дата приема внешнее место работы', 'Внешнее совместительство'])

HR['Количество ставок'] = HR['Количество ставок'].apply(convert)
HR_PPS = HR[(HR['Категория персонала'] == 'ППС')]
HR_PPS_NR = HR[(HR['Категория персонала'] == 'ППС') | (HR['Категория персонала'] == 'НР')]
cat_HR = HR[(HR['Вид занятости'] == 'Основное место работы') | (HR['Вид занятости'] == 'Внешнее совместительство')][['ФИО', 'Категория персонала']].drop_duplicates()
cat_HR = cat_HR[~((cat_HR['ФИО'] == 'Евсеев Александр Владимирович') & (cat_HR['Категория персонала'] == 'ППС'))]
HR_NR_ONLY = HR_PPS_NR[~(HR_PPS_NR['ФИО'].isin(HR_PPS_NR[(HR_PPS_NR['Категория персонала'] == 'ППС')]['ФИО']))]
HR_NOT_PPS_NR = HR[~((HR['Категория персонала'] == 'ППС') | (HR['Категория персонала'] == 'НР'))]

temp_PPS_NR = HR_PPS_NR.groupby(['ФИО']).sum()[['Количество ставок']].reset_index(level=0)
temp_NOT_PPS_NR = HR_NOT_PPS_NR.groupby(['ФИО']).sum()[['Количество ставок']].reset_index(level=0)
HR_PPS_NR = HR_PPS_NR.drop(['Категория персонала', 'Вид занятости', 'Количество ставок'], axis=1)

positions = HR_PPS[['ФИО', 'Должность']]
positions['Ранг'] = HR_PPS['Должность'].map({'Ассистент': 0, 
                                               'Преподаватель': 1, 
                                               'Старший преподаватель': 1,
                                               'Доцент': 2,
                                               'Декан факультета': 2,
                                               'Профессор': 3})
positions = positions.groupby('ФИО', group_keys=False).apply(lambda x: x.loc[x['Ранг'].idxmax()]).reset_index(drop=True).drop(columns=['Ранг'])
HR_PPS['Должность'] = HR_PPS['Должность'].map({'Ассистент': 0, 
                                               'Преподаватель': 1, 
                                               'Старший преподаватель': 1,
                                               'Доцент': 2,
                                               'Декан факультета': 2,
                                               'Профессор': 3})
HR_PPS = HR_PPS.groupby('ФИО', group_keys=False).apply(lambda x: x.loc[x['Должность'].idxmax()]).reset_index(drop=True)
HR_PPS = HR_PPS.merge(positions, left_on='ФИО', right_on='ФИО', how='left')
HR_PPS['Должность_x'] = HR_PPS['Должность_y']
HR_PPS = HR_PPS.drop(columns=['Должность_y']).rename(columns={"Должность_x": "Должность"})

HR_PPS_NR = pd.concat([HR_PPS, HR_NR_ONLY]).reset_index(drop=True).sort_values(by=['ФИО']).drop(columns=['Вид занятости', 'Категория персонала', 'Количество ставок'])

HR_PPS_NR = HR_PPS_NR.merge(temp_PPS_NR, left_on='ФИО', right_on='ФИО', how='left')
HR_PPS_NR = HR_PPS_NR.merge(temp_NOT_PPS_NR, left_on='ФИО', right_on='ФИО', how='left')
HR_PPS_NR = HR_PPS_NR.rename(columns={"Количество ставок_y": "Количество ставок НЕ ППС и НР", 
                                      "Количество ставок_x": "Количество ставок ППС и НР"})
HR_PPS_NR = HR_PPS_NR.fillna(value={'Количество ставок ППС и НР': 0,
                                    'Количество ставок НЕ ППС и НР': 0})

HR_PPS_NR = HR_PPS_NR.merge(cat_HR, left_on='ФИО', right_on='ФИО', how='left').rename(columns={"Категория персонала": "Категория персонала (по осн. или внеш.)"})

HR_PPS_NR = HR_PPS_NR.merge(external_work, left_on='ФИО', right_on='ФИО', how='left').fillna(value={'Внешнее совместительство': 'нет'})

HR_PPS_NR_not_ranked = HR_PPS_NR.copy()


HR_PPS_NR['Должность'] = HR_PPS_NR['Должность'].map({'Ассистент': 0, 
                                                     'Преподаватель': 1, 
                                                     'Старший преподаватель': 1,
                                                     'Руководитель центра': 1,
                                                     'Заведующий лабораторией химического анализа': 1,
                                                     'Доцент': 2,
                                                     'Декан факультета': 2,
                                                     'Ведущий научный сотрудник': 2,
                                                     'Профессор': 3, 
                                                     'Главный научный сотрудник ': 3})
HR_PPS_NR['Ученое звание'] = HR_PPS_NR['Ученое звание'].map({np.nan: 0, 
                                                             'Доцент': 1, 
                                                             'Старший научный сотрудник': 1,
                                                             'Профессор': 2})
HR_PPS_NR['Ученая степень'] = HR_PPS_NR['Ученая степень'].map({np.nan: 0, 
                                                               'Кандидат наук': 1, 
                                                               'Доктор наук': 2})
HR_PPS_NR['Внешнее совместительство'] = HR_PPS_NR['Внешнее совместительство'].map({'нет': 0, 
                                                                                   'есть': 1, 
                                                                                   'есть_специалист': 1, 
                                                                                   'есть_начальник_директор_руководитель': 2})
HR_PPS_NR = HR_PPS_NR.fillna(0)
HR_PPS_NR_ranked = HR_PPS_NR.drop(columns=['Подразделение', 'Категория персонала (по осн. или внеш.)']).copy()

C:\Users\Stepan\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Stepan\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<font size="6">Добавление стажа работы</font>

In [5]:
experience = pd.read_excel('Стаж 2019.xls')
experience.columns = experience.iloc[5]
experience.columns.name = None
experience = experience.iloc[6:].reset_index(drop=True)
experience = experience.rename(columns={'Сотрудник': 'ФИО'})
experience['ФИО'] = experience['ФИО'].str.title()
experience['ФИО'] = experience['ФИО'].replace('\s+', ' ', regex=True)
experience['ФИО'] = experience['ФИО'].replace('ё', 'е', regex=True)
experience['ФИО'] = experience['ФИО'].str.strip()
experience['Вид стажа'] = experience['Вид стажа'].replace('\s+', ' ', regex=True)
experience['Вид стажа'] = experience['Вид стажа'].str.strip()
experience = experience[['ФИО', 'Вид стажа', 'Лет', 'Месяцев', 'Дней']]
experience['Лет'] = experience['Лет'].str[:].astype('float64').fillna(0)
experience['Месяцев'] = experience['Месяцев'].str[:].astype('float64').fillna(0)
experience['Дней'] = experience['Дней'].str[:].astype('float64').fillna(0)
experience['Лет'] = (experience['Лет'] + ((experience['Месяцев'] + experience['Дней'] / 30) / 12)).round(1)
experience = experience.drop(columns=['Месяцев', 'Дней'])
print(experience['Вид стажа'].unique())
choose_exp = ['Общий стаж', 'Стаж в УрГУПС', 'Стаж работы в должности', 'Педагогический стаж', 'Непрерывный на предприятии']
experience = experience[experience['Вид стажа'].isin(choose_exp)]
experience = experience.groupby(['Вид стажа', 'ФИО']).sum().unstack(['Вид стажа']).reset_index()
experience.columns = experience.columns.get_level_values(1)
experience = experience.rename(columns={experience.columns[0]: "ФИО"})
experience.columns.name = None

HR_PPS_NR_not_ranked = HR_PPS_NR_not_ranked.merge(experience, left_on='ФИО', right_on='ФИО', how='left')

['Непрерывный стаж' 'Общий стаж' 'Стаж в УрГУПС' 'Стаж работы в должности'
 'Страховой стаж для оплаты больничных листов' 'Педагогический стаж'
 'Непрерывный на предприятии'
 'Стаж для оплаты больничных листов с учетом нестраховых периодов (с 2010 года)'
 'Общий научно-педагогический стаж работы'
 'Стаж по территориальным условиям МКС']


<font size="20">Данные по НИРС</font>

In [7]:
NIRS = pd.read_excel('Объемы работы ППС по НИРС за 1 и 2 квартал 2019.xlsx', sheet_name='Лист1').fillna(0)
NIRS['ФИО'] = NIRS['ФИО'].str.title()
NIRS['ФИО'] = NIRS['ФИО'].replace('\s+', ' ', regex=True)
NIRS['ФИО'] = NIRS['ФИО'].replace('ё', 'е', regex=True)
NIRS['ФИО'] = NIRS['ФИО'].str.strip()
NIRS = NIRS.groupby(['ФИО']).sum().reset_index(level=0)

<font size="20">Данные по корпоративной почте</font>

In [8]:
mails = pd.read_excel('Список рассылки Опроса по корпоративной почте.xlsx', sheet_name='Используют КорпПОчту', names=['ФИО'], header=None)
mails['ФИО'] = mails['ФИО'].str.title()
mails['ФИО'] = mails['ФИО'].replace('\s+', ' ', regex=True)
mails['ФИО'] = mails['ФИО'].replace('ё', 'е', regex=True)
mails['ФИО'] = mails['ФИО'].str.strip()
mails['Почта'] = pd.Series('есть', index=mails.index)

<font size="20">Данные по Вовлеченность в НИР (Responses)</font>

In [9]:
responses = pd.read_excel('Вовлеченность в НИР (Responses)_управКоммуникация.xlsx', sheet_name='Form Responses 1')
responses = responses.rename(columns={"1.  Укажите, пожалуйста, Ваши Фамилию Имя Отчество (полностью)": "ФИО"})
responses['ФИО'] = responses['ФИО'].str.title()
responses['ФИО'] = responses['ФИО'].replace('\s+', ' ', regex=True)
responses['ФИО'] = responses['ФИО'].replace('ё', 'е', regex=True)
responses['ФИО'] = responses['ФИО'].str.strip()
responses = responses.sort_values(by=['ФИО', 'Timestamp'])
responses = responses.drop_duplicates(subset=['ФИО'], keep='last')
#responses = responses.drop(columns=['Timestamp'])

<font size="20">Данные по нагрузке ППС</font>

In [10]:
busy_PPS = pd.read_excel('.\\Время ППС\\Нагрузка ППС 2019-2020 уч.г.xlsx', sheet_name='Лист1')
busy_PPS['ФИО'] = busy_PPS['ФИО'].str.title()
busy_PPS['ФИО'] = busy_PPS['ФИО'].replace('\s+', ' ', regex=True)
busy_PPS['ФИО'] = busy_PPS['ФИО'].replace('ё', 'е', regex=True)
busy_PPS['ФИО'] = busy_PPS['ФИО'].str.strip()

<font size="20">Данные по публикациям</font>

In [11]:
articles = pd.read_excel('Публикации ППС выгрузка окт 2019.xlsx', sheet_name='Данные')
articles = articles[articles['Выгрузка'] == 'от 21.10'].drop_duplicates(subset=['ФИО', 'Наименование'])
articles = articles.loc[:, ['ФИО', 'Журнал', 'Статусы', 'Год',
                            'Зарубежный', 'ИмпактФактор', 'Количество соавторов']]
articles['ФИО'] = articles['ФИО'].str.title()
articles['ФИО'] = articles['ФИО'].replace('\s+', ' ', regex=True)
articles['ФИО'] = articles['ФИО'].replace('ё', 'е', regex=True)
articles['ФИО'] = articles['ФИО'].str.strip()
articles = articles.loc[articles['Год'] >= 2015]

articles['Журнал'] = articles['Журнал'].str.lower()
values = {'Журнал': 'без названия'}
articles = articles.fillna(value=values)
articles['ИмпактФактор'] = articles['ИмпактФактор'].apply(convert_impact)
articles["ИмпактФактор"] = articles.groupby("Журнал")["ИмпактФактор"].transform(lambda x: x.fillna(x.mean()))
values = {'ИмпактФактор': 0.}
articles = articles.fillna(value=values)

articles['Публикация статьи'] = np.where((articles['Журнал']=='транспорт урала') |
                                         (articles['Журнал']=='инновационный транспорт') |
                                         (articles['Журнал']=='вестник ургупс') |
                                         (articles['Журнал']=='инновационный транспорт - 2016: специализация железных дорог'), 
                                         'ургупс', 'не_ургупс')
articles['Статусы'] = articles['Статусы'].map({np.nan:                    np.nan, 
                                               'РИНЦ':                    'РИНЦ', 
                                               'ВАК':                     'ВАК',
                                               'ВАК, РИНЦ':               'ВАК',
                                               'ВАК, РИНЦ, RSCI':         'ВАК',
                                               'ВАК, RSCI':               'ВАК',
                                               'Scopus':                  'Scopus_or_WoS',
                                               'РИНЦ, Scopus, WoS':       'Scopus_or_WoS',
                                               'ВАК, РИНЦ, Scopus':       'Scopus_or_WoS', 
                                               'ВАК, РИНЦ, Scopus, WoS':  'Scopus_or_WoS', 
                                               'РИНЦ, Scopus':            'Scopus_or_WoS',
                                               'РИНЦ, WoS':               'Scopus_or_WoS',
                                               'ВАК, RSCI, WoS':          'Scopus_or_WoS',
                                               'WoS':                     'Scopus_or_WoS',
                                               'ВАК, RSCI, Scopus':       'Scopus_or_WoS',
                                               'ВАК, РИНЦ, RSCI, WoS':    'Scopus_or_WoS',
                                               'ВАК, РИНЦ, RSCI, Scopus': 'Scopus_or_WoS'})

articles = articles.sort_values(by=['Журнал', 'Год'])
articles['Статусы'] = articles.groupby(['Журнал'], sort=False)['Статусы'].apply(lambda x: x.bfill())
articles = articles.sort_values(by=['ФИО'])
values = {'Статусы': 'нет'}
articles = articles.fillna(value=values)

temp = articles.groupby(['ФИО', 'Статусы', 'Зарубежный', 'Публикация статьи'])['ФИО'].agg('count').to_frame('count').reset_index()
temp_status = temp.pivot_table(index=['ФИО'], columns=['Статусы'], values=['count'], aggfunc='sum').fillna(0)
temp_foreign = temp.pivot_table(index=['ФИО'], columns=['Зарубежный'], values=['count'], aggfunc='sum').fillna(0)
temp_usurt = temp.pivot_table(index=['ФИО'], columns=['Публикация статьи'], values=['count'], aggfunc='sum').fillna(0)

temp_status.columns = temp_status.columns.droplevel(0)
temp_status.reset_index(inplace=True)
temp_foreign.columns = temp_foreign.columns.droplevel(0)
temp_foreign.reset_index(inplace=True)
temp_usurt.columns = temp_usurt.columns.droplevel(0)
temp_usurt.reset_index(inplace=True)
temp_foreign = temp_foreign.rename(columns={'Да': 'кол-во Иностранных', 
                                            'Нет': 'кол-во Отечественных'})
temp_status = temp_status.rename(columns={'нет': 'без статуса'})
temp_usurt = temp_usurt.rename(columns={'не_ургупс': 'кол-во НЕ в ургупс', 
                                        'ургупс': 'кол-во в ургупс'})

temp = articles[['ФИО', 'ИмпактФактор', 'Количество соавторов']]
temp_mean = temp.groupby('ФИО')['ИмпактФактор', 'Количество соавторов'].mean().reset_index()
temp_mean = temp_mean.rename(columns={'ИмпактФактор': 'Средний ИмпактФактор', 
                                      'Количество соавторов': 'Среднее количество соавторов'})
temp_min = temp.groupby('ФИО')['Количество соавторов'].min().reset_index()
temp_min = temp_min.rename(columns={'Количество соавторов': 'Минимальное количество соавторов'})
temp_max = temp.groupby('ФИО')['ИмпактФактор'].max().reset_index()
temp_max = temp_max.rename(columns={'ИмпактФактор': 'Максимальный ИмпактФактор'})

temp = temp_min.merge(temp_max, left_on='ФИО', right_on='ФИО', how='left')
temp = temp_mean.merge(temp, left_on='ФИО', right_on='ФИО', how='left')
temp = temp_usurt.merge(temp, left_on='ФИО', right_on='ФИО', how='left')
temp = temp_status.merge(temp, left_on='ФИО', right_on='ФИО', how='left')
temp = temp_foreign.merge(temp, left_on='ФИО', right_on='ФИО', how='left')

articles = temp.drop(columns=['Минимальное количество соавторов']).copy()

<font size="20">Данные по времени преподавания</font>

In [12]:
def extract_teach_time(name):
    teach_time = pd.read_excel(name)
    teach_time = teach_time.drop(columns=['СотрудникКодКадры', 'Пар', 'учебных часов']).rename(columns={'Сотрудник': 'ФИО', 
                                                                                                        'Астрономических часов': 'Время преподавания'})
    teach_time['ФИО'] = teach_time['ФИО'].str.title()
    teach_time['ФИО'] = teach_time['ФИО'].replace('\s+', ' ', regex=True)
    teach_time['ФИО'] = teach_time['ФИО'].replace('ё', 'е', regex=True)
    teach_time['ФИО'] = teach_time['ФИО'].str.strip()
    return teach_time

In [13]:
teach_time_oct = extract_teach_time('.\\Время ППС\\Сотрудники пары окт2019.xls')
teach_time_nov = extract_teach_time('.\\Время ППС\\Сотрудники пары ноя2019.xls')
teach_time_dec = extract_teach_time('.\\Время ППС\\Сотрудники пары дек2019.xls')
teach_time = teach_time_oct.merge(teach_time_nov, on='ФИО', how='outer', suffixes=('_окт2019', '_ноя2019'))
teach_time = teach_time.merge(teach_time_dec, on='ФИО', how='outer').rename(columns={'Время преподавания': 'Время преподавания_дек2019'})

<font size="20">Данные по времени работы в УрГУПС</font>

In [14]:
def extract_work_time(name):
    work_time = pd.read_excel(name, sheet_name = 'Отработанное время за месяц')
    work_time.columns = work_time.iloc[7]
    work_time.columns.name = None
    work_time = work_time.iloc[8:]
    work_time = work_time.iloc[0:-4].dropna(how='all')
    work_time = work_time.loc[:, work_time.columns.notnull()].drop(columns=['№'])
    work_time['Фамилия, инициалы, должность (специальность, профессия)'] = work_time["Фамилия, инициалы, должность (специальность, профессия)"].str.split("\\n", n = 2, expand = True)[0]
    work_time = work_time.rename(columns={'Фамилия, инициалы, должность (специальность, профессия)': 'ФИО'})
    work_time['ФИО'] = work_time['ФИО'].str.title()
    work_time['ФИО'] = work_time['ФИО'].replace('\s+', ' ', regex=True)
    work_time['ФИО'] = work_time['ФИО'].replace('ё', 'е', regex=True)
    work_time['ФИО'] = work_time['ФИО'].replace('Белоусов Виталий Витальевич', 'Белоусов Виталий Виталиевич', regex=True)
    work_time['ФИО'] = work_time['ФИО'].str.strip()
    work_time = work_time.iloc[:,:-1].reset_index(drop=True)
    if 'X' in work_time.columns:
        work_time = work_time.drop(columns=['X'])
    work_time = eval_mean_sum(work_time)
    return work_time

In [15]:
work_time_oct = extract_work_time('.\\Время ППС\\Отработанное время за октябрь все работники.xlsx')
work_time_nov = extract_work_time('.\\Время ППС\\Отработанное время за ноябрь все работники.xlsx')
work_time_dec = extract_work_time('.\\Время ППС\\Отработанное время за декабрь все работники.xlsx')
work_time = work_time_oct.merge(work_time_nov, on='ФИО', how='outer', suffixes=('_окт2019', '_ноя2019'))
work_time = work_time.merge(work_time_dec, on='ФИО', how='outer').rename(columns={'Ср продолжительность в УрГУПС': 'Ср продолжительность в УрГУПС_дек2019',
                                                                                  'Общая продолжительность в УрГУПС': 'Общая продолжительность в УрГУПС_дек2019'})

C:\Users\Stepan\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: Mean of empty slice
C:\Users\Stepan\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: Mean of empty slice


<font size="20">Данные по анализу работы в УрГУПС</font>

In [16]:
work_analysis = pd.read_excel('.\\Время ППС\\общая информация о работе сотрудников (с пропусками)_анализ.xlsx', sheet_name='Sheet1')[['ФИО', 'Расписание, часов', 'Разность Р-П', 'Пребывание, часов', 'Относительное пребывание', 'Среднее в день', 'Классификация ППС по времени работы']]
work_analysis['ФИО'] = work_analysis['ФИО'].str.title()
work_analysis['ФИО'] = work_analysis['ФИО'].replace('\s+', ' ', regex=True)
work_analysis['ФИО'] = work_analysis['ФИО'].replace('ё', 'е', regex=True)
work_analysis['ФИО'] = work_analysis['ФИО'].str.strip()

<font size="20">Командировки и конференции</font>

In [17]:
business_trip = pd.read_excel('Расходы подотчетных лиц.xls')
business_trip = business_trip[business_trip['Назначение аванса'] == 'Командировочные расходы'][['Сотрудник', 'Комментарий']].sort_values(by=['Сотрудник']).reset_index(drop=True)
business_trip['Комментарий'] = business_trip['Комментарий'].apply(convert_business_trip)
business_trip = business_trip.groupby(['Сотрудник', 'Комментарий'])['Сотрудник'].agg('count').to_frame('count').reset_index()
business_trip = business_trip.pivot_table(index=['Сотрудник'], columns=['Комментарий'], values=['count'], aggfunc='sum').fillna(0)
business_trip.columns = business_trip.columns.droplevel(0)
business_trip.reset_index(inplace=True)
business_trip = business_trip.rename(columns={'Сотрудник': 'ФИО', 
                                              'Иностр': 'Кол-во зарубежных командировок', 
                                              'Москва': 'Кол-во командировок в Москву',
                                              'Россия': 'Кол-во командировок в России (кроме Москвы)'})
business_trip['ФИО'] = business_trip['ФИО'].str.title()
business_trip['ФИО'] = business_trip['ФИО'].replace('\s+', ' ', regex=True)
business_trip['ФИО'] = business_trip['ФИО'].replace('ё', 'е', regex=True)
business_trip['ФИО'] = business_trip['ФИО'].str.strip()
business_trip.columns.name = None

In [18]:
conferences = pd.read_excel('Конференции 13-11-2019.xlsx')
conferences = conferences[['ФИО', 'Наименование', 'МестоПроведения']]
conferences = conferences[~(conferences['МестоПроведения'].isnull())]
conferences = conferences[['ФИО', 'МестоПроведения']]
conferences['МестоПроведения'] = conferences['МестоПроведения'].apply(convert_conferences)
conferences = conferences.groupby(['ФИО', 'МестоПроведения'])['ФИО'].agg('count').to_frame('count').reset_index()
conferences = conferences.pivot_table(index=['ФИО'], columns=['МестоПроведения'], values=['count'], aggfunc='sum').fillna(0)
conferences.columns = conferences.columns.droplevel(0)
conferences.reset_index(inplace=True)
conferences = conferences.rename(columns={'Сотрудник': 'ФИО', 
                                          'НЕ в ургупс': 'Кол-во конференций НЕ в УрГУПС', 
                                          'в ургупс': 'Кол-во конференций в УрГУПС'})
conferences['ФИО'] = conferences['ФИО'].str.title()
conferences['ФИО'] = conferences['ФИО'].replace('\s+', ' ', regex=True)
conferences['ФИО'] = conferences['ФИО'].replace('ё', 'е', regex=True)
conferences['ФИО'] = conferences['ФИО'].str.strip()
conferences.columns.name = None

<font size="20">Индекс Хирша</font>

In [19]:
index_h = pd.read_excel('Индекс Хирша всех ППС.xlsx', sheet_name='Sheet1').drop(columns=['№'])
index_h['ФИО'] = index_h['ФИО'].str.title()
index_h['ФИО'] = index_h['ФИО'].replace('\s+', ' ', regex=True)
index_h['ФИО'] = index_h['ФИО'].replace('ё', 'е', regex=True)
index_h['ФИО'] = index_h['ФИО'].str.strip()

<font size="20">Слияние данных</font>

In [21]:
final = HR_PPS_NR_not_ranked.merge(mails, left_on='ФИО', right_on='ФИО', how='left').fillna(value={'Почта': 'нет'})
final = final.merge(responses, on='ФИО', how='outer')
final = final.merge(busy_PPS, left_on='ФИО', right_on='ФИО', how='left')
final = final.merge(NIRS, left_on='ФИО', right_on='ФИО', how='left')
final = final.merge(articles, left_on='ФИО', right_on='ФИО', how='left')
final = final.merge(business_trip, left_on='ФИО', right_on='ФИО', how='left')
final = final.merge(conferences, left_on='ФИО', right_on='ФИО', how='left')
final = final.merge(index_h, left_on='ФИО', right_on='ФИО', how='left')
final = final.merge(teach_time, left_on='ФИО', right_on='ФИО', how='left')
final = final.merge(work_time, left_on='ФИО', right_on='ФИО', how='left')
final = final.merge(work_analysis, left_on='ФИО', right_on='ФИО', how='left')

In [ ]:
#final#.set_index('ФИО').to_excel("общий массив данных ППС 22.12.19.xlsx")

In [31]:
#final['uuid'] = [uuid.uuid4() for _ in range(len(final.index))]

In [35]:
#final[['ФИО', 'uuid']].set_index('ФИО').to_excel("сотруник-uuid.xlsx")